# Develop and test the Lang-graph agents as a standalone
#### The purpose of this notebook is to build, test, and assess the lang-graph agents with dummy Google API results
#### It also runs the politifact data.
#### Author: Michael Denton
#### Published Date: April 15, 2024

In [1]:

import pandas as pd
import os
import operator
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser, CommaSeparatedListOutputParser
from langchain_community.llms import Ollama
from typing import TypedDict, Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain.tools import Tool
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain.agents import AgentType, initialize_agent, load_tools

from typing import List
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import json


text = "The economy has grown by 4 percent in the last year, and Joe Biden is the reason. Joe Biden has grown hundreds of thousands of jobs. Despite being 81 years old, Joe Biden is still doing amazing work."


llm = Ollama(model="mistral-openorca")

"""
From a large text entry, supports the extraction of claims from the text
"""

# llm = Ollama(model="llama2")

# Define the output parser
string_parser = StrOutputParser()
# json_parser = JsonOutputParser()
list_parser = CommaSeparatedListOutputParser()

claims_agent_messages = [
    ("system", "[INST]You are tasked with extracting the verifiable claims from the user query."),
    ("system", "Each claim should exist as a unique immutable string in a python list without any extras, like this:\n['first claim, 'second claim', 'third claim']\n"),
    ("system", "The domain focus is politics."),
    ("system", "One statement per claim."),
    ("user", "The user query:\n{input}\n"),
    ("system", "[/INST]")
]

judge_agent_messages = [
    ("system", "[INST]You are tasked with judging whether the following claims are contained within the text."),
    ("system", "If these Claims, Facts, or Opinions are not in the original text, you will answer 'no'. If they are, answer 'yes'"),
    ("system", "The claims are:\n{extracted_claims}\n"),
    ("system", "These were claims from the following text:\n{input}\n"),
    ("system", "Are the claims are contained within the text? Answer only 'yes' or 'no' with no other text.[/INST]")
]

feedback_agent_messages = [
    ("system", "Agents have assessed text for verifiable claims"),
    ("system", "A judge has compared these to the original text and decided that the extracted claims are not sufficient based on the original text."),
    ("system", "Given the analysis and the original text, provide instructions to improve the output."),
    ("user", "The verifiable claims are:\n{extracted_claims}\n"),
    ("user", "The origin text is:\n{input}\n"),
    ("user", "What changes should the agents implement?"),
]


feedback_prompt = "You have previously done this before, and a judge has suggested the following notes to improve your answer: {reinforcer_notes}"


# claims_agent = claims_agent_messages | llm | list_parser
# judge_agent = judge_agent_messages | llm | list_parser

# Define the agent state
class AgentState(TypedDict):
    input_text: str
    claims_to_check: list[str]
    baseline: bool
    apiversion: int
    reinforcer_notes: str
    judge_output: str
    iteration: int
    max_iterations: int
    final_output: list
    responses: list

def input(state):
    input_text = state["input_text"]
    
    return {"input_text": input_text}

def is_one_sentence(state):
    """
    Checks if one sentence, if it is output go to search.
    Otherwise we'll parse out facts in each sentence
    """

    input_text = state["input_text"]
    if (len([val for val in str(input_text).split('.') if val != '']) == 1):
        return 'go_to_search'
    return 'extract_claims_from_text'


def extract_claims(state):
    input_text = state["input_text"]
    if state.get("reinforcer_notes"):
        claims_agent_messages.append(("system", feedback_prompt))
    reinforcer_notes = state.get("reinforcer_notes", "")

    claims_prompt = ChatPromptTemplate.from_messages(claims_agent_messages)
    claims_agent = claims_prompt | llm | list_parser
    response = claims_agent.invoke({"input": input_text, "reinforcer_notes": reinforcer_notes})
    print(response)
    # The format comes back as a list within a list
    if 'list' in str(type(response)).lower():
        
        if len(response) == 1:
            new_response = response[0]
        else:
            new_response = response

        if 'list' in str(type(new_response)).lower():
            if len(new_response) > 1:
                formatted_responses = []
                for val in new_response:
                    val = val.replace('[','').replace(']','').replace('<|im_end|>','')
                    if '\n' in val:
                        vals = val.split('\n')
                        formatted_responses = formatted_responses + vals
                    else:
                        formatted_responses.append(val)
                response = formatted_responses
            

    return {"claims_to_check": response}

def extract_judgement(state):
    formatted_extraction = state.get("claims_to_check")
    input = state.get("input_text")

    judge_prompt = ChatPromptTemplate.from_messages(judge_agent_messages)
    judge_agent = judge_prompt | llm | string_parser
    verdict = judge_agent.invoke({"input": input, "extracted_claims": formatted_extraction})
    print(verdict)
    if 'yes' in str(verdict).lower():
        verdict = 'yes'
    else:
        verdict = 'no'

    return {"judge_output": verdict}

def feedback_on_incorrect_extractions(state):
    formatted_extraction = state.get("claims_to_check")
    input = state.get("input_text")
    instructor_prompt = ChatPromptTemplate.from_messages(feedback_agent_messages)
    instructor_agent = instructor_prompt | llm | string_parser
    instructions = instructor_agent.invoke({"input": input, "extracted_claims":formatted_extraction})
    return {"reinforcer_notes": instructions}                                                   

def judge_happy(state):
    judges_verdict = state.get("judge_output")
    judges_verdict = judges_verdict.strip().replace("\n", "").lower()
    if judges_verdict == 'yes':
        return 'continue'
    return 'feedback'


def google_search_agent(state):
    """
    AgentType
    """
    claims_to_check = state["claims_to_check"] 

    if 'list' not in str(type(claims_to_check)).lower():
        claims_to_check = [claims_to_check]

    responses = []
    for clm in claims_to_check:
        responses.append(google_search_agent_call(stuff=clm, llm=llm))

    return {"responses": responses} 

def print_output(state):
    responses = state.get("responses")

    return {"final_output": responses}

graph = StateGraph(AgentState)

# Define nodes in our graph

graph.add_node("input", input)
graph.add_node("claims_agent", extract_claims)
graph.add_node("judge_agent", extract_judgement)
graph.add_node("feedback_agent", feedback_on_incorrect_extractions)
graph.add_node("search_agent", google_search_agent)
graph.add_node("output", print_output)

graph.add_conditional_edges(
    "input", 
    is_one_sentence, 
    {
        "go_to_search": "search_agent",
        "extract_claims_from_text": "claims_agent"
    }
)
graph.add_edge('claims_agent', 'judge_agent')
graph.add_conditional_edges(
    "judge_agent",
    judge_happy,
    {
        "feedback": "feedback_agent",
        "continue": "search_agent"
    }
)
graph.add_edge('feedback_agent', 'claims_agent')
graph.add_edge("search_agent", "output")
graph.add_edge('output', END)

graph.set_entry_point("input")

verify = graph.compile()

inputs = {"input_text": text}



In [4]:
verify.get_graph().print_ascii()

                        +-----------+                          
                        | __start__ |                          
                        +-----------+                          
                              *                                
                              *                                
                              *                                
                          +-------+                            
                          | input |                            
                          +-------+                            
                              *                                
                              *                                
                              *                                
                  +-----------------------+                    
                  | input_is_one_sentence |                    
                  +-----------------------+*                   
                      ***               

In [10]:
import pprint
out = verify.invoke({"input_text": "I am a single sentence."})
pprint.pp(out)

{'input_text': 'I am a single sentence.',
 'claims_to_check': None,
 'baseline': None,
 'apiversion': None,
 'reinforcer_notes': None,
 'judge_output': None,
 'iteration': None,
 'max_iterations': None,
 'final_output': [{'statement': None,
                   'judgement': 'False',
                   'justification': 'Dummy,',
                   'process_time': None,
                   'information': None,
                   'message': 'Successfully returned judgement'}],
 'responses': [{'statement': None,
                'judgement': 'False',
                'justification': 'Dummy,',
                'process_time': None,
                'information': None,
                'message': 'Successfully returned judgement'}]}


In [66]:
pprint.pp(out['final_output'])

[{'statement': "'Economy growth: 4 percent in the last year'",
  'judgement': 'False',
  'justification': 'Dummy,',
  'process_time': None,
  'information': None,
  'message': 'Successfully returned judgement'},
 {'statement': "'Joe Biden: responsible for economic growth and job creation'",
  'judgement': 'False',
  'justification': 'Dummy,',
  'process_time': None,
  'information': None,
  'message': 'Successfully returned judgement'},
 {'statement': "'Joe Biden: despite his age",
  'judgement': 'False',
  'justification': 'Dummy,',
  'process_time': None,
  'information': None,
  'message': 'Successfully returned judgement'},
 {'statement': "doing amazing work'",
  'judgement': 'False',
  'justification': 'Dummy,',
  'process_time': None,
  'information': None,
  'message': 'Successfully returned judgement'}]


In [34]:

out = None
for output in verify.stream(inputs):
    # stream() yields dictionaries with output keyed by node name
    for key, value in output.items():
        print(f"Output from node '{key}':")
        print("---")
        print(value)
    print("\n---\n")

# for output in verify.stream(inputs):
#     print(output)

Output from node 'input':
---
{'input_text': 'The economy has grown by 4 percent in the last year, and Joe Biden is the reason. Joe Biden has grown hundreds of thousands of jobs. Despite being 81 years old, Joe Biden is still doing amazing work.'}

---

Output from node 'claims_agent':
---
{'claims_to_check': ['1. The economy has grown by 4 percent in the last year.\n2. Joe Biden is the reason for the economic growth.\n3. Joe Biden has created hundreds of thousands of jobs.\n4. Despite being 81 years old', 'Joe Biden is still doing amazing work.<|im_end|>']}

---

 yes<|im_end|>
Output from node 'judge_agent':
---
{'judge_output': 'yes'}

---

Output from node 'search_agent':
---
{'responses': [{'statement': '1. The economy has grown by 4 percent in the last year.\n2. Joe Biden is the reason for the economic growth.\n3. Joe Biden has created hundreds of thousands of jobs.\n4. Despite being 81 years old', 'judgement': 'False', 'justification': 'Dummy,', 'process_time': None, 'informatio

In [8]:
def google_search_agent_call(stuff=None, llm=None):

    tmp = {
        'statement': stuff,
        'judgement': 'False',
        'justification': 'Dummy,',
        'process_time': None,
        'information': None,
        'message': 'Successfully returned judgement'
    }

    return tmp